# Initial AI Agent PPO w/o
NN (working) MR2 Drag Disc

In [1]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [2]:
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import numpy as np
import gymnasium
from time import sleep

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 384
my_config["act_buf_len"] = 4
my_config["reset_act_buf"] = True
my_config["benchmark"] = False
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'discreteAccel' : True,
  'accelAndBrake' : False,
  'discSteer' : True,
  'contAccelOnly' : False,
  'discAccelOnly' : False,
  'modelMode': 13,
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 1, # 0 is HS, 1 is 400m
  'carChoice' : 0, # 0 is MR2, 1 is Supra, 2 is Civic
  'rewardMode' : "complex"
}

In [3]:
def env_creator(env_config):
    env = gymnasium.make("real-time-gym-v1", config=env_config)
    return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [4]:
import ray
ray.shutdown()
ray.init()

2023-10-20 22:14:57,083	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.9
Ray version:,2.4.0
Dashboard:,http://127.0.0.1:8265


In [5]:
from ray.rllib.algorithms.ppo import PPOConfig
config = (
    PPOConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        enable_connectors=True,
        batch_mode="truncate_episodes",
        #batch_mode="completed_episodes",
        )
    .framework(
        framework="torch",
        )
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
        train_batch_size=384,
        lr=0.0001,
        )
)

In [6]:
import json

class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        try:
            json.dumps(obj)
        except TypeError:
            return "Not serializable object!"
        return obj

config_dict = config.to_dict()

print(json.dumps(config_dict, sort_keys=True, indent=4, cls=CustomEncoder))

{
    "_disable_action_flattening": false,
    "_disable_execution_plan_api": true,
    "_disable_preprocessor_api": false,
    "_enable_learner_api": false,
    "_enable_rl_module_api": false,
    "_fake_gpus": false,
    "_learner_hps": "Not serializable object!",
    "_tf_policy_handles_more_than_one_loss": false,
    "_validate_exploration_conf_and_rl_modules": true,
    "action_space": null,
    "actions_in_input_normalized": false,
    "always_attach_evaluation_results": false,
    "auto_wrap_old_gym_envs": true,
    "batch_mode": "truncate_episodes",
    "callbacks": "Not serializable object!",
    "checkpoint_trainable_policies_only": false,
    "clip_actions": false,
    "clip_param": 0.3,
    "clip_rewards": null,
    "compress_observations": false,
    "create_env_on_driver": false,
    "custom_eval_function": null,
    "custom_resources_per_worker": {},
    "delay_between_worker_restarts_s": 60.0,
    "disable_env_checking": true,
    "eager_max_retraces": 20,
    "eager_tr

In [7]:
algo = config.build()

2023-10-20 22:15:02,839	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=13968) GT Real Time instantiated
(RolloutWorker pid=13968) GT AI Server instantiated for rtgym
(RolloutWorker pid=13968) starting up on localhost port 9999
(RolloutWorker pid=13968) Waiting for a connection
(RolloutWorker pid=13968) Connection from ('127.0.0.1', 51346)
(RolloutWorker pid=13968) J:\git\TensorFlowPSX\Py\dragSpaced.csv
(RolloutWorker pid=13968) Discrete Accel or Brake (cannot left foot) and Discrete Steering 2
(RolloutWorker pid=13968) 0 1


2023-10-20 22:15:31,380	INFO trainable.py:172 -- Trainable.setup took 28.557 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [8]:
algo.get_policy().model

FullyConnectedNetwork(
  (_logits): SlimFC(
    (_model): Sequential(
      (0): Linear(in_features=256, out_features=4, bias=True)
    )
  )
  (_hidden_layers): Sequential(
    (0): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=10518, out_features=256, bias=True)
        (1): Tanh()
      )
    )
    (1): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): Tanh()
      )
    )
  )
  (_value_branch_separate): Sequential(
    (0): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=10518, out_features=256, bias=True)
        (1): Tanh()
      )
    )
    (1): SlimFC(
      (_model): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): Tanh()
      )
    )
  )
  (_value_branch): SlimFC(
    (_model): Sequential(
      (0): Linear(in_features=256, out_features=1, bias=True)
    )
  )
)

In [ ]:
N = 3000

for n in range(N):
    result = algo.train()
    print("Loop: ", n)
    if n % 10 == 0:
        print("Saved", n)
        algo.save()
        
algo.save()

(RolloutWorker pid=13968) reset triggered
(RolloutWorker pid=13968) 0 8 72
(RolloutWorker pid=13968) reload save for track : 72
(RolloutWorker pid=13968) reset triggered
(RolloutWorker pid=13968) 0 8 72
(RolloutWorker pid=13968) reload save for track : 72


(RolloutWorker pid=13968) C:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.17076450000084975
(RolloutWorker pid=13968)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  0
Saved 0


(RolloutWorker pid=13968) C:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 1.0299184000000068
(RolloutWorker pid=13968)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=13968) reset triggered
(RolloutWorker pid=13968) 0 8 72
(RolloutWorker pid=13968) reload save for track : 72


...

(RolloutWorker pid=13968) C:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.9711937998363283
(RolloutWorker pid=13968)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=13968) C:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.052747799178177956
(RolloutWorker pid=13968)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=13968) reset triggered
(RolloutWorker pid=13968) 0 8 72
(RolloutWorker pid=13968) reload save for track : 72


(RolloutWorker pid=13968) C:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.24092939986439887
(RolloutWorker pid=13968)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  1760
Saved 1760


(RolloutWorker pid=13968) C:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.9676569998409832
(RolloutWorker pid=13968)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


#Works!

In [ ]:
#from ray.rllib.algorithms.algorithm import Algorithm
#algo = Algorithm.from_checkpoint("C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-10-14_22-11-44ayp9l_kd/checkpoint_000881")

In [ ]:
# result = algo.train() #single try